In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import ast
from importable_functions.get_global_dataframe_limits import *
from importable_functions.polynomial_fitting import *
import random
import os

In [ ]:
# ********************
# ****** IMPORT ******
# ********************
df_k729_2022_cuid = pd.read_csv('./datasets/k729_2022_cuid.csv')
df_k729_2022_cuid_grouped = pd.read_csv('./datasets/k729_2022_cuid_grouped.csv')
df_k729_2022_cuid_grouped['x'] = df_k729_2022_cuid_grouped['x'].apply(lambda x: ast.literal_eval(x))
df_k729_2022_cuid_grouped['y'] = df_k729_2022_cuid_grouped['y'].apply(lambda y: ast.literal_eval(y))
df_k729_2022_cuid_grouped['vx'] = df_k729_2022_cuid_grouped['vx'].apply(lambda vx: ast.literal_eval(vx))
df_k729_2022_cuid_grouped['vy'] = df_k729_2022_cuid_grouped['vy'].apply(lambda vy: ast.literal_eval(vy))
df_k729_2022_cuid_grouped['v'] = df_k729_2022_cuid_grouped['v'].apply(lambda v: ast.literal_eval(v))

df_k733_2020_cuid = pd.read_csv('./datasets/k733_2020_cuid.csv')
df_k733_2020_cuid_grouped = pd.read_csv('./datasets/k733_2020_cuid_grouped.csv')
df_k733_2020_cuid_grouped['x'] = df_k733_2020_cuid_grouped['x'].apply(lambda x: ast.literal_eval(x))
df_k733_2020_cuid_grouped['y'] = df_k733_2020_cuid_grouped['y'].apply(lambda y: ast.literal_eval(y))

df_k733_2018_cuid = pd.read_csv('./datasets/k733_2018_cuid.csv')
df_k733_2018_cuid_grouped = pd.read_csv('./datasets/k733_2018_cuid_grouped.csv')
df_k733_2018_cuid_grouped['x'] = df_k733_2018_cuid_grouped['x'].apply(lambda x: ast.literal_eval(x))
df_k733_2018_cuid_grouped['y'] = df_k733_2018_cuid_grouped['y'].apply(lambda y: ast.literal_eval(y))
df_k733_2018_cuid_grouped['vx'] = df_k733_2018_cuid_grouped['vx'].apply(lambda vx: ast.literal_eval(vx))
df_k733_2018_cuid_grouped['vy'] = df_k733_2018_cuid_grouped['vy'].apply(lambda vy: ast.literal_eval(vy))

In [ ]:
x = np.array(df_k729_2022_cuid_grouped['x'][0])
y = np.array(df_k729_2022_cuid_grouped['y'][0])
params, covariance = curve_fit(quadratic, x, y)
y_fit = quadratic(x, *params)

In [ ]:
limits = get_global_dataframe_limits(df_k729_2022_cuid, ['x','y'])

In [ ]:
i = 69
x_i = np.array(df_k729_2022_cuid_grouped['x'][i])
y_fit_i = quadratic(x_i, *params)
y_i = np.array(df_k729_2022_cuid_grouped['y'][i])
plt.scatter(x_i, y_i, label='Data')
plt.xlim(limits['x_min'], limits['x_max'])
plt.ylim(limits['y_min'], limits['y_max'])
plt.plot(x_i, y_fit_i, color='red', label=f'Fit: $y={params[0]:.2f}x^2 + {params[1]:.2f}x + {params[2]:.2f}$')
plt.legend()
plt.show()

In [ ]:
group = group_D
params, covariance = fit_polynomial_for_group(df_k729_2022_cuid_grouped,group,quadratic)
i = random.randint(0,len(group)) # specific x,y for checking w the plot
i = group[i]
x_i = np.array(df_k729_2022_cuid_grouped['x'][i])
y_i = np.array(df_k729_2022_cuid_grouped['y'][i])

y_i_fit = quadratic(x_i, *params)

plt.scatter(x_i, y_i, label='True Path')
plt.xlim(limits['x_min'], limits['x_max'])
plt.ylim(limits['y_min'], limits['y_max'])
plt.plot(x_i, y_i_fit, color='red', label=f'Fit: $y={params[0]:.2f}x^2 + {params[1]:.2f}x + {params[2]:.2f}$')
plt.legend()
plt.title('Lane A model fit')
plt.show()

In [ ]:
group_A = [8,13,15,45,46,47]
group_B = [18,79,80,81,83]
group_C = [16,21,23,25,74,72]
group_D = [2,4]
group_1 = [3,28,30,34,98,105]
group_2 = [7,14,35,41,42,48]
group_3 = [0,1,5,11,27,29,32,33]
group_5 = [9,10,136,135,70,71]
group_7 = [19,22,26,57,76,77,84]
group_9 = [24,31,36,37,38,50,51,52]
all_groups = [group_A,group_B,group_C,group_D,group_1,group_2,group_3,group_5,group_7,group_9]
all_groups_names = ['Group A', 'Group B', 'Group C', 'Group D', 'Group 1', 'Group 2', 'Group 3', 'Group 5', 'Group 7', 'Group 9']

In [ ]:
# train model for each group
# for every vehicle: 
#                   fit y values 
#                   plot fit vs real values
#                   calculate R_sq
for j, group in enumerate(all_groups):
    # fit model
    params, covariance = fit_polynomial_for_group(df_k729_2022_cuid_grouped,group,quadratic)

    # create dir to save images in 
    path_dir_save_imgs = os.path.expanduser(f'~/Pictures/thesis/k729_2022/vehicle_tracks_polynomial_fitting/{all_groups_names[j]}/')
    try:
        os.makedirs(path_dir_save_imgs)
        print(f'Created directory {path_dir_save_imgs}')
    except Exception as e:
        print(f'Directory creation went wrong w/ error: {type(e)}')

    # plot predicted vs real path for each vehicle
    for i in range(len(df_k729_2022_cuid_grouped['track_id'])):
        
        x = np.array(df_k729_2022_cuid_grouped['x'][i])
        y = np.array(df_k729_2022_cuid_grouped['y'][i])
        y_hat = quadratic(x, *params)

        r_sq = get_R_squared(y,y_hat)

        plt.scatter(x, y, label='True Path')
        plt.xlim(limits['x_min'], limits['x_max'])
        plt.ylim(limits['y_min'], limits['y_max'])
        plt.plot(x, y_hat, color='red', label=f'Fit: $y={params[0]:.2f}x^2 + {params[1]:.2f}x + {params[2]:.2f}$')
        plt.legend()
        plt.suptitle(f'Predicted Path for Vehicle #{i} with model from {all_groups_names[j]}', fontweight='bold')
        plt.title(f'With $R^2={r_sq}$')
        plt.savefig(f'{path_dir_save_imgs}/polynomial_fit_track_id_{i}')
        plt.clf()

In [ ]:
group = group_3
params, covariance = fit_polynomial_for_group(df_k729_2022_cuid_grouped,group,quadratic)

x = np.array(df_k729_2022_cuid_grouped['x'][40])
y = np.array(df_k729_2022_cuid_grouped['y'][40])
y_hat = quadratic(x,*params)
y_mean = np.mean(y)
y_mean_hat = np.mean(y_hat)
print(f'y: {y}')
print(f'y_hat: {y_hat}')
print(f'y_mean: {y_mean}')
print(f'y_mean_hat: {y_mean_hat}')